In [2]:
import numpy as np
import random

grid_size = 5
gamma = 0.95
actions = ['up', 'down', 'left', 'right']
action_effects = {
    'up': (-1, 0),
    'down': (1, 0),
    'left': (0, -1),
    'right': (0, 1)
}
special_states = {
    (0, 0): ('blue', 5, (4, 0)),
    (0, 4): ('green', 2.5, [(4, 4), (4, 0)]),
    (4, 0): ('yellow', 0, None),
    (4, 4): ('red', 0, None)
}



In [3]:
import numpy as np

# Define the grid size and discount factor
grid_size = 5
gamma = 0.95

# Define the possible actions and their effects
actions = ['up', 'down', 'left', 'right']
action_effects = {
    'up': (-1, 0),
    'down': (1, 0),
    'left': (0, -1),
    'right': (0, 1)
}

# Define the special states and their effects
special_states = {
    (0, 0): ('blue', 5, (4, 0)),
    (0, 4): ('green', 2.5, [(4, 4), (4, 0)]),
    (4, 0): ('yellow', 0, None),
    (4, 4): ('red', 0, None)
}

def get_next_state(state, action):
    x, y = state
    dx, dy = action_effects[action]
    new_state = (max(0, min(grid_size - 1, x + dx)), max(0, min(grid_size - 1, y + dy)))
    return new_state

def build_system():
    A = np.zeros((grid_size * grid_size, grid_size * grid_size))
    b = np.zeros(grid_size * grid_size)
    
    for i in range(grid_size):
        for j in range(grid_size):
            state = (i, j)
            state_idx = i * grid_size + j
            if state in special_states:
                if special_states[state][0] == 'blue':
                    reward = special_states[state][1]
                    next_state = special_states[state][2]
                    next_state_idx = next_state[0] * grid_size + next_state[1]
                    A[state_idx, state_idx] = 1
                    b[state_idx] = reward + gamma * (np.ones(4) / 4).sum() * A[next_state_idx, state_idx]
                elif special_states[state][0] == 'green':
                    reward = special_states[state][1]
                    next_states = special_states[state][2]
                    next_state_idxs = [ns[0] * grid_size + ns[1] for ns in next_states]
                    A[state_idx, state_idx] = 1
                    b[state_idx] = reward + gamma * 0.5 * (A[next_state_idxs[0], state_idx] + A[next_state_idxs[1], state_idx])
                else:
                    A[state_idx, state_idx] = 1
            else:
                for action in actions:
                    next_state = get_next_state(state, action)
                    next_state_idx = next_state[0] * grid_size + next_state[1]
                    A[state_idx, state_idx] += -1 / len(actions)
                    A[state_idx, next_state_idx] += gamma / len(actions)
    return A, b

A, b = build_system()
values = np.linalg.solve(A, b)
value_function_explicit = values.reshape((grid_size, grid_size))

print("Value function (Explicit):")
print(value_function_explicit)


Value function (Explicit):
[[5.         2.70014423 1.82231019 1.7490911  2.5       ]
 [2.54799512 1.84657392 1.4013395  1.29319283 1.44479288]
 [1.33383146 1.12556922 0.93829989 0.84979895 0.84535272]
 [0.60873665 0.62053358 0.57402868 0.50125543 0.41943533]
 [0.         0.30443842 0.35687398 0.26728621 0.        ]]


In [4]:
def policy_evaluation(policy, gamma, theta=1e-6):
    value_function = np.zeros((grid_size, grid_size))
    
    while True:
        delta = 0
        for i in range(grid_size):
            for j in range(grid_size):
                state = (i, j)
                v = value_function[state]
                new_v = 0
                if state in special_states:
                    if special_states[state][0] == 'blue':
                        reward = special_states[state][1]
                        next_state = special_states[state][2]
                        new_v = reward + gamma * value_function[next_state]
                    elif special_states[state][0] == 'green':
                        reward = special_states[state][1]
                        next_states = special_states[state][2]
                        new_v = reward + gamma * 0.5 * (value_function[next_states[0]] + value_function[next_states[1]])
                else:
                    for action in actions:
                        next_state = get_next_state(state, action)
                        new_v += 1 / len(actions) * (-0.2 + gamma * value_function[next_state])
                value_function[state] = new_v
                delta = max(delta, abs(v - new_v))
        if delta < theta:
            break
    return value_function

# Initial policy: Equal probability for all actions
policy = np.ones((grid_size, grid_size, len(actions))) / len(actions)

value_function_iterative = policy_evaluation(policy, gamma)

print("Value function (Iterative Policy Evaluation):")
print(value_function_iterative)


Value function (Iterative Policy Evaluation):
[[ 5.          1.37799231  0.14677657  0.42693862  2.5       ]
 [ 1.2258432   0.1194069  -0.49159788 -0.43397486  0.12263994]
 [-0.34170217 -0.76736815 -1.05999062 -1.04313909 -0.8301821 ]
 [-0.71341583 -1.1066341  -1.31890936 -1.22591278 -0.90271797]
 [ 0.         -1.01771451 -1.31866084 -1.05486708  0.        ]]


In [5]:
import numpy as np

# Define the grid size and discount factor
grid_size = 5
gamma = 0.95

# Define the possible actions and their effects
actions = ['up', 'down', 'left', 'right']
action_effects = {
    'up': (-1, 0),
    'down': (1, 0),
    'left': (0, -1),
    'right': (0, 1)
}

# Define the special states and their effects
special_states = {
    (0, 0): ('blue', 5, (4, 0)),
    (0, 4): ('green', 2.5, [(4, 4), (4, 0)]),
    (4, 0): ('yellow', 0, None),
    (4, 4): ('red', 0, None)
}

def get_next_state(state, action):
    x, y = state
    dx, dy = action_effects[action]
    new_state = (max(0, min(grid_size - 1, x + dx)), max(0, min(grid_size - 1, y + dy)))
    return new_state

def value_iteration(gamma, theta=1e-6):
    value_function = np.zeros((grid_size, grid_size))
    
    while True:
        delta = 0
        new_value_function = value_function.copy()
        
        for i in range(grid_size):
            for j in range(grid_size):
                state = (i, j)
                v = value_function[state]
                
                if state in special_states:
                    if special_states[state][0] == 'blue':
                        reward = special_states[state][1]
                        next_state = special_states[state][2]
                        new_v = reward + gamma * value_function[next_state]
                    elif special_states[state][0] == 'green':
                        reward = special_states[state][1]
                        next_states = special_states[state][2]
                        new_v = reward + gamma * 0.5 * (value_function[next_states[0]] + value_function[next_states[1]])
                    else:
                        new_v = 0
                else:
                    new_v = max(
                        (1 / len(actions)) * (-0.2 + gamma * value_function[get_next_state(state, action)]) for action in actions
                    )
                
                new_value_function[state] = new_v
                delta = max(delta, abs(v - new_v))
        
        value_function = new_value_function
        if delta < theta:
            break
    
    return value_function

value_function_value_iteration = value_iteration(gamma)

print("Value function (Value Iteration):")
print(value_function_value_iteration)


Value function (Value Iteration):
[[ 5.00000000e+00  1.13750000e+00  2.20156250e-01  5.43750000e-01
   2.50000000e+00]
 [ 1.13750000e+00  2.20156250e-01  2.28710937e-03  7.91406250e-02
   5.43750000e-01]
 [ 2.20156250e-01  2.28710937e-03 -4.94568115e-02 -3.12041016e-02
   7.91406250e-02]
 [ 2.28710937e-03 -4.94568115e-02 -6.17459927e-02 -5.74109741e-02
  -3.12041016e-02]
 [ 0.00000000e+00 -5.00000000e-02 -6.18750000e-02 -5.00000000e-02
   0.00000000e+00]]


In [6]:
# Identify the states with the highest values
max_explicit = np.unravel_index(np.argmax(value_function_explicit), value_function_explicit.shape)
max_policy = np.unravel_index(np.argmax(value_function_iterative), value_function_iterative.shape)
max_value_iter = np.unravel_index(np.argmax(value_function_value_iteration), value_function_value_iteration.shape)

print(f"Highest value (explicit): {max_explicit} with value {value_function_explicit[max_explicit]}")
print(f"Highest value (policy evaluation): {max_policy} with value {value_function_iterative[max_policy]}")
print(f"Highest value (value iteration): {max_value_iter} with value {value_function_value_iteration[max_value_iter]}")


Highest value (explicit): (0, 0) with value 5.0
Highest value (policy evaluation): (0, 0) with value 5.0
Highest value (value iteration): (0, 0) with value 5.0


In [7]:
import numpy as np

grid_size = 5
gamma = 0.95

actions = ['up', 'down', 'left', 'right']
action_effects = {
    'up': (-1, 0),
    'down': (1, 0),
    'left': (0, -1),
    'right': (0, 1)
}

special_states = {
    (0, 0): ('blue', 5, (4, 0)),
    (0, 4): ('green', 2.5, [(4, 4), (4, 0)]),
    (4, 0): ('yellow', 0, None),
    (4, 4): ('red', 0, None)
}

def initialize_gridworld(grid_size, special_states):
    rewards = np.zeros((grid_size, grid_size))
    transitions = np.empty((grid_size, grid_size), dtype=object)
    
    for state, (color, reward, next_states) in special_states.items():
        rewards[state] = reward
        transitions[state] = next_states
    
    return rewards, transitions

def get_next_state(state, action, grid_size, transitions):
    x, y = state
    dx, dy = action_effects[action]
    new_state = (x + dx, y + dy)
    
    if new_state in transitions:
        if isinstance(transitions[new_state], list):
            new_state = transitions[new_state][np.random.choice(len(transitions[new_state]))]
        else:
            new_state = transitions[new_state]
    
    new_x, new_y = new_state
    if new_x < 0 or new_x >= grid_size or new_y < 0 or new_y >= grid_size:
        return state
    return new_state

def get_reward(state, rewards):
    return rewards[state]


In [8]:
import numpy as np

grid_size = 5
gamma = 0.95

actions = ['up', 'down', 'left', 'right']
action_effects = {
    'up': (-1, 0),
    'down': (1, 0),
    'left': (0, -1),
    'right': (0, 1)
}

special_states = {
    (0, 0): ('blue', 5, (4, 0)),
    (0, 4): ('green', 2.5, [(4, 4), (4, 0)]),
    (4, 0): ('yellow', 0, None),
    (4, 4): ('red', 0, None)
}

def initialize_gridworld(grid_size, special_states):
    rewards = np.zeros((grid_size, grid_size))
    transitions = {}
    
    for state, (color, reward, next_states) in special_states.items():
        rewards[state] = reward
        transitions[state] = next_states
    
    return rewards, transitions

def is_terminal(state):
    return state in special_states and special_states[state][2] is None

def get_next_state(state, action, grid_size, transitions):
    x, y = state
    dx, dy = action_effects[action]
    new_state = (x + dx, y + dy)
    
    if new_state in transitions:
        if isinstance(transitions[new_state], list):
            new_state = transitions[new_state][np.random.choice(len(transitions[new_state]))]
        else:
            new_state = transitions[new_state]
    
    if new_state is None:
        return state

    new_x, new_y = new_state
    if new_x < 0 or new_x >= grid_size or new_y < 0 or new_y >= grid_size:
        return state
    return new_state

def get_reward(state, rewards):
    return rewards[state]

### 1. Solving the Bellman Optimality Equation

def bellman_optimality(grid_size, rewards, transitions, gamma=0.95, theta=1e-6):
    value_function = np.zeros((grid_size, grid_size))
    policy = np.zeros((grid_size, grid_size), dtype=int)
    
    while True:
        delta = 0
        new_values = np.copy(value_function)
        for x in range(grid_size):
            for y in range(grid_size):
                state = (x, y)
                if is_terminal(state):
                    continue
                
                action_values = []
                for action in actions:
                    next_state = get_next_state(state, action, grid_size, transitions)
                    reward = get_reward(next_state, rewards)
                    action_values.append(reward + gamma * value_function[next_state])
                
                new_values[state] = max(action_values)
                policy[state] = np.argmax(action_values)
                delta = max(delta, abs(value_function[state] - new_values[state]))
        
        value_function = new_values
        if delta < theta:
            break
    
    return value_function, policy

# Initialize gridworld
rewards, transitions = initialize_gridworld(grid_size, special_states)
value_function_bellman, policy_bellman = bellman_optimality(grid_size, rewards, transitions)
print("Value Function (Bellman Optimality):\n", value_function_bellman)
print("Optimal Policy (Bellman Optimality):\n", policy_bellman)

### 2. Policy Iteration with Iterative Policy Evaluation

def policy_evaluation(policy, grid_size, rewards, transitions, gamma=0.95, theta=1e-6):
    value_function = np.zeros((grid_size, grid_size))
    
    while True:
        delta = 0
        new_values = np.copy(value_function)
        for x in range(grid_size):
            for y in range(grid_size):
                state = (x, y)
                if is_terminal(state):
                    continue
                
                action = actions[policy[state]]
                next_state = get_next_state(state, action, grid_size, transitions)
                reward = get_reward(next_state, rewards)
                new_values[state] = reward + gamma * value_function[next_state]
                delta = max(delta, abs(value_function[state] - new_values[state]))
        
        value_function = new_values
        if delta < theta:
            break
    
    return value_function

def policy_iteration(grid_size, rewards, transitions, gamma=0.95, theta=1e-6):
    policy = np.zeros((grid_size, grid_size), dtype=int)
    
    while True:
        value_function = policy_evaluation(policy, grid_size, rewards, transitions, gamma, theta)
        
        policy_stable = True
        for x in range(grid_size):
            for y in range(grid_size):
                state = (x, y)
                if is_terminal(state):
                    continue
                
                old_action = policy[state]
                action_values = []
                for action in actions:
                    next_state = get_next_state(state, action, grid_size, transitions)
                    reward = get_reward(next_state, rewards)
                    action_values.append(reward + gamma * value_function[next_state])
                
                policy[state] = np.argmax(action_values)
                if old_action != policy[state]:
                    policy_stable = False
        
        if policy_stable:
            break
    
    return value_function, policy

# Initialize gridworld
rewards, transitions = initialize_gridworld(grid_size, special_states)
value_function_policy_iter, policy_policy_iter = policy_iteration(grid_size, rewards, transitions)
print("Value Function (Policy Iteration):\n", value_function_policy_iter)
print("Optimal Policy (Policy Iteration):\n", policy_policy_iter)

### 3. Policy Improvement with Value Iteration

def value_iteration_with_policy(grid_size, rewards, transitions, gamma=0.95, theta=1e-6):
    value_function = np.zeros((grid_size, grid_size))
    policy = np.zeros((grid_size, grid_size), dtype=int)
    
    while True:
        delta = 0
        new_values = np.copy(value_function)
        for x in range(grid_size):
            for y in range(grid_size):
                state = (x, y)
                if is_terminal(state):
                    continue
                
                action_values = []
                for action in actions:
                    next_state = get_next_state(state, action, grid_size, transitions)
                    reward = get_reward(next_state, rewards)
                    action_values.append(reward + gamma * value_function[next_state])
                
                new_values[state] = max(action_values)
                policy[state] = np.argmax(action_values)
                delta = max(delta, abs(value_function[state] - new_values[state]))
        
        value_function = new_values
        if delta < theta:
            break
    
    return value_function, policy

# Initialize gridworld
rewards, transitions = initialize_gridworld(grid_size, special_states)
value_function_value_iter, policy_value_iter = value_iteration_with_policy(grid_size, rewards, transitions)
print("Value Function (Value Iteration with Policy Improvement):\n", value_function_value_iter)
print("Optimal Policy (Value Iteration with Policy Improvement):\n", policy_value_iter)


Value Function (Bellman Optimality):
 [[99.99998127  0.          0.          0.         49.99999064]
 [ 0.          0.          0.          0.          0.        ]
 [ 0.          0.          0.          0.          0.        ]
 [ 0.          0.          0.          0.          0.        ]
 [ 0.          0.          0.          0.          0.        ]]
Optimal Policy (Bellman Optimality):
 [[0 0 0 0 0]
 [0 0 0 0 0]
 [0 0 0 0 0]
 [0 0 0 0 0]
 [0 0 0 0 0]]
Value Function (Policy Iteration):
 [[99.99998127  0.          0.          0.         49.99999064]
 [ 0.          0.          0.          0.          0.        ]
 [ 0.          0.          0.          0.          0.        ]
 [ 0.          0.          0.          0.          0.        ]
 [ 0.          0.          0.          0.          0.        ]]
Optimal Policy (Policy Iteration):
 [[0 0 0 0 0]
 [0 0 0 0 0]
 [0 0 0 0 0]
 [0 0 0 0 0]
 [0 0 0 0 0]]
Value Function (Value Iteration with Policy Improvement):
 [[99.99998127  0.          0.

### PART 1

In [9]:
import numpy as np

# Define grid size
size = 5
gamma = 0.95

# Initialize rewards
rewards = np.zeros((size, size))
# Special rewards
rewards[0, 1] = 5  # Blue square
rewards[1, 2] = 2.5  # Green square

# Initialize transitions
transitions = np.zeros((size, size, 4, 2), dtype=int)
for i in range(size):
    for j in range(size):
        transitions[i, j, 0] = [max(i-1, 0), j]  # Up
        transitions[i, j, 1] = [min(i+1, size-1), j]  # Down
        transitions[i, j, 2] = [i, max(j-1, 0)]  # Left
        transitions[i, j, 3] = [i, min(j+1, size-1)]  # Right

# Special transitions
transitions[0, 1] = [[4, 1], [4, 1], [4, 1], [4, 1]]  # Blue to Red
transitions[1, 2] = [[2, 3], [2, 3], [2, 3], [2, 3]]  # Green to Yellow/Red with 0.5 prob each


In [10]:
from scipy.linalg import solve

# Define the state space
num_states = size * size
states = [(i, j) for i in range(size) for j in range(size)]

# Define the transition and reward matrices
P = np.zeros((num_states, num_states))
R = np.zeros(num_states)

for idx, (i, j) in enumerate(states):
    for action in range(4):
        next_state = transitions[i, j, action]
        next_idx = states.index(tuple(next_state))
        P[idx, next_idx] += 0.25  # Equal probability for all actions
        R[idx] += 0.25 * rewards[next_state[0], next_state[1]]

# Solve the Bellman equation
I = np.eye(num_states)
V = solve(I - gamma * P, R)

# Reshape V to 2D grid
V_grid = V.reshape((size, size))
print("Value Function (Solving Bellman Equations):")
print(V_grid)


Value Function (Solving Bellman Equations):
[[4.57697642 1.38417783 4.42820117 2.96990739 2.30973636]
 [3.4701911  4.11528987 1.96803432 2.79702832 2.13582561]
 [2.44887355 2.61039621 2.6211121  2.07161507 1.75035965]
 [1.78158568 1.80586643 1.75463693 1.5540897  1.41213503]
 [1.46508772 1.45702929 1.40687673 1.30514855 1.22924733]]


In [11]:
def iterative_policy_evaluation(V, theta=1e-6):
    while True:
        delta = 0
        for i in range(size):
            for j in range(size):
                v = V[i, j]
                V[i, j] = 0
                for action in range(4):
                    next_state = transitions[i, j, action]
                    V[i, j] += 0.25 * (rewards[next_state[0], next_state[1]] + gamma * V[next_state[0], next_state[1]])
                delta = max(delta, abs(v - V[i, j]))
        if delta < theta:
            break
    return V

V_iterative = np.zeros((size, size))
V_iterative = iterative_policy_evaluation(V_iterative)
print("Value Function (Iterative Policy Evaluation):")
print(V_iterative)


Value Function (Iterative Policy Evaluation):
[[1.78653436 0.39079888 2.48556839 1.11118596 0.57909762]
 [1.50973274 2.94279006 1.06883064 1.61401261 0.85916259]
 [0.97223796 1.52659738 1.71350234 1.1250855  0.73014412]
 [0.56464093 0.79924935 0.86265544 0.67954397 0.50002787]
 [0.3030647  0.41136758 0.4399386  0.37346964 0.29162969]]


In [12]:
def value_iteration(V, theta=1e-6):
    while True:
        delta = 0
        for i in range(size):
            for j in range(size):
                v = V[i, j]
                V[i, j] = max([
                    rewards[transitions[i, j, action][0], transitions[i, j, action][1]] + gamma * V[transitions[i, j, action][0], transitions[i, j, action][1]]
                    for action in range(4)
                ])
                delta = max(delta, abs(v - V[i, j]))
        if delta < theta:
            break
    return V

V_value_iteration = np.zeros((size, size))
V_value_iteration = value_iteration(V_value_iteration)
print("Value Function (Value Iteration):")
print(V_value_iteration)


Value Function (Value Iteration):
[[22.10246707 18.00259757 22.10246769 20.9973443  19.94747709]
 [20.99734371 22.10246769 18.00259757 19.94747709 18.95010323]
 [19.94747653 20.9973443  19.94747709 18.95010323 18.00259807]
 [18.9501027  19.94747709 18.95010323 18.00259807 17.10246817]
 [18.00259757 18.95010323 18.00259807 17.10246817 16.24734476]]


In [13]:
def policy_iteration(V, policy, theta=1e-6):
    stable_policy = False
    while not stable_policy:
        V = iterative_policy_evaluation(V)
        stable_policy = True
        for i in range(size):
            for j in range(size):
                old_action = policy[i, j]
                action_values = [
                    rewards[transitions[i, j, action][0], transitions[i, j, action][1]] + gamma * V[transitions[i, j, action][0], transitions[i, j, action][1]]
                    for action in range(4)
                ]
                best_action = np.argmax(action_values)
                policy[i, j] = best_action
                if old_action != best_action:
                    stable_policy = False
    return policy, V

policy = np.random.choice(4, size=(size, size))
V_policy_iteration = np.zeros((size, size))
policy, V_policy_iteration = policy_iteration(V_policy_iteration, policy)
print("Optimal Policy (Policy Iteration):")
print(policy)
print("Value Function (Policy Iteration):")
print(V_policy_iteration)


Optimal Policy (Policy Iteration):
[[3 0 2 2 2]
 [3 0 0 2 2]
 [3 0 0 2 2]
 [0 0 0 0 0]
 [0 0 0 0 0]]
Value Function (Policy Iteration):
[[1.78653462 0.3907992  2.48556875 1.11118626 0.57909784]
 [1.50973314 2.94279059 1.06883123 1.61401307 0.85916293]
 [0.97223838 1.52659791 1.71350289 1.12508597 0.73014447]
 [0.56464127 0.79924977 0.86265587 0.67954434 0.50002814]
 [0.3030649  0.41136783 0.43993885 0.37346986 0.29162985]]


In [14]:
def policy_improvement(V):
    policy = np.zeros((size, size), dtype=int)
    for i in range(size):
        for j in range(size):
            action_values = [
                rewards[transitions[i, j, action][0], transitions[i, j, action][1]] + gamma * V[transitions[i, j, action][0], transitions[i, j, action][1]]
                for action in range(4)
            ]
            policy[i, j] = np.argmax(action_values)
    return policy

policy_from_value_iteration = policy_improvement(V_value_iteration)
print("Optimal Policy (Value Iteration):")
print(policy_from_value_iteration)


Optimal Policy (Value Iteration):
[[3 0 2 2 2]
 [3 0 0 0 0]
 [3 0 2 0 0]
 [3 0 0 0 0]
 [3 0 0 0 0]]


### PART 2

In [15]:
import numpy as np

# Initialize the grid size
size = 5

# Initialize rewards with new conditions
rewards = np.full((size, size), -0.2)
rewards[0, 1] = 5  # Blue square
rewards[0, 4] = 2.5  # Green square

# Define terminal states (black squares)
terminal_states = [(2, 4), (0, 4)]

# Set rewards for terminal states
for (i, j) in terminal_states:
    rewards[i, j] = 0

# Initialize transitions with terminal states
def init_transitions_with_terminals():
    transitions = np.zeros((size, size, 4, 2), dtype=int)
    for i in range(size):
        for j in range(size):
            if (i, j) not in terminal_states:
                transitions[i, j, 0] = [max(i-1, 0), j]  # Up
                transitions[i, j, 1] = [min(i+1, size-1), j]  # Down
                transitions[i, j, 2] = [i, max(j-1, 0)]  # Left
                transitions[i, j, 3] = [i, min(j+1, size-1)]  # Right
    # Special transitions
    transitions[0, 1] = [[4, 1], [4, 1], [4, 1], [4, 1]]  # Blue to Red
    transitions[1, 2] = [[2, 3], [2, 3], [2, 3], [2, 3]]  # Green to Yellow/Red with 0.5 prob each
    return transitions

transitions = init_transitions_with_terminals()

In [16]:
def monte_carlo_with_exploring_starts(episodes=500, gamma=0.95, epsilon=0.1):
    V = np.zeros((size, size))
    returns = {state: [] for state in [(i, j) for i in range(size) for j in range(size)]}
    policy = np.random.choice(4, size=(size, size))

    for _ in range(episodes):
        # Generate an episode with exploring starts
        start_state = (np.random.randint(0, size), np.random.randint(0, size))
        while start_state in terminal_states:
            start_state = (np.random.randint(0, size), np.random.randint(0, size))
        state = start_state
        episode = []
        
        while state not in terminal_states:
            action = np.random.choice(4)
            next_state = transitions[state[0], state[1], action]
            reward = rewards[next_state[0], next_state[1]]
            episode.append((state, action, reward))
            state = tuple(next_state)
        
        G = 0
        for t in range(len(episode)-1, -1, -1):
            state, action, reward = episode[t]
            G = gamma * G + reward
            if not (state, action) in [(x[0], x[1]) for x in episode[:t]]:
                returns[state].append(G)
                V[state] = np.mean(returns[state])
                action_values = [
                    rewards[transitions[state[0], state[1], a][0], transitions[state[0], state[1], a][1]] + gamma * V[transitions[state[0], state[1], a][0], transitions[state[0], state[1], a][1]]
                    for a in range(4)
                ]
                policy[state[0], state[1]] = np.argmax(action_values)
                
    return policy, V

policy_mc_exploring, V_mc_exploring = monte_carlo_with_exploring_starts()
print("Optimal Policy (Monte Carlo with Exploring Starts):")
print(policy_mc_exploring)
print("Value Function (Monte Carlo with Exploring Starts):")
print(V_mc_exploring)


Optimal Policy (Monte Carlo with Exploring Starts):
[[3 0 2 2 1]
 [0 0 0 0 0]
 [0 0 3 3 1]
 [0 0 0 0 0]
 [0 0 3 0 0]]
Value Function (Monte Carlo with Exploring Starts):
[[ 0.81363842 -2.53918239  0.408184   -0.32191055  0.        ]
 [-0.74738658 -0.26458762 -1.27193994 -0.94761911 -0.47766531]
 [-1.78651188 -1.53589315 -1.7031521  -1.05957375  0.        ]
 [-2.3874413  -2.21964436 -2.11975938 -1.73386187 -1.24289107]
 [-2.61843231 -2.45097416 -2.28158504 -2.05246246 -1.81610831]]


In [17]:
def monte_carlo_e_soft(episodes=500, gamma=0.95, epsilon=0.1, alpha=0.1):
    V = np.zeros((size, size))
    returns = {state: [] for state in [(i, j) for i in range(size) for j in range(size)]}
    policy = np.random.choice(4, size=(size, size))

    for _ in range(episodes):
        state = (np.random.randint(0, size), np.random.randint(0, size))
        while state in terminal_states:
            state = (np.random.randint(0, size), np.random.randint(0, size))
        episode = []
        visited_state_actions = set()
        
        while state not in terminal_states:
            if np.random.rand() < epsilon:
                action = np.random.choice(4)
            else:
                action = policy[state[0], state[1]]
            next_state = transitions[state[0], state[1], action]
            reward = rewards[next_state[0], next_state[1]]
            episode.append((state, action, reward))
            state = tuple(next_state)
        
        G = 0
        for t in range(len(episode)-1, -1, -1):
            state, action, reward = episode[t]
            G = gamma * G + reward
            if (state, action) not in visited_state_actions:
                visited_state_actions.add((state, action))
                returns[state].append(G)
                V[state] = V[state] + alpha * (G - V[state])
                action_values = [
                    rewards[transitions[state[0], state[1], a][0], transitions[state[0], state[1], a][1]] + gamma * V[transitions[state[0], state[1], a][0], transitions[state[0], state[1], a][1]]
                    for a in range(4)
                ]
                policy[state[0], state[1]] = np.argmax(action_values)
                
    return policy, V

policy_mc_e_soft, V_mc_e_soft = monte_carlo_e_soft()
print("Optimal Policy (Monte Carlo ε-soft):")
print(policy_mc_e_soft)
print("Value Function (Monte Carlo ε-soft):")
print(V_mc_e_soft)

Optimal Policy (Monte Carlo ε-soft):
[[3 0 2 1 2]
 [0 0 0 0 2]
 [0 0 3 0 1]
 [0 0 0 2 2]
 [0 0 0 0 2]]
Value Function (Monte Carlo ε-soft):
[[ 4.55453277  0.87880265  4.15035268  2.93442971  0.        ]
 [ 3.97064728  1.50082547 -0.55465907  2.71091054  2.0463414 ]
 [ 2.21388036  1.78529323  1.40163956  2.05937958  0.        ]
 [ 1.11424808  1.0185867   0.62041245  1.64811808  0.03401989]
 [ 0.8227976   0.40633061 -0.3318594  -0.33342701 -0.51245531]]


In [22]:
import numpy as np
import random

# Gridworld setup
grid_size = 5
gamma = 0.95  # Discount factor
actions = [(0, 1), (0, -1), (1, 0), (-1, 0)]  # Up, Down, Right, Left
epsilon = 0.1  # Exploration factor
alpha = 0.1  # Learning rate

# Initialize rewards for all states with -0.2 default (white-to-white transitions)
rewards_2 = np.full((grid_size, grid_size), -0.2)
rewards_2[0, 1] = 5   # Blue square reward
rewards_2[0, 4] = 2.5 # Green square reward

# Terminal states setup
is_terminal = np.zeros((grid_size, grid_size), dtype=bool)
is_terminal[2, 4] = True  # Terminal state at location [2,4]
is_terminal[4, 0] = True  # Terminal state at location [4,0]

def get_reward(i, j, next_i, next_j):
    """Calculate and return the next state positions and reward."""
    if next_i < 0 or next_i >= grid_size or next_j < 0 or next_j >= grid_size:
        return i, j, -0.5  # Penalty for stepping off the grid
    if is_terminal[next_i, next_j]:
        return next_i, next_j, 0  # No further rewards when moving into a terminal state
    return next_i, next_j, rewards_2[next_i, next_j]  # Return the actual reward from the rewards matrix

def monte_carlo_epsilon_soft(num_episodes=500, epsilon=0.1, gamma=0.95, alpha=0.1):
    Q = np.zeros((grid_size, grid_size, len(actions)))
    returns_count = np.zeros((grid_size, grid_size, len(actions)))

    for episode in range(num_episodes):
        i, j = random.randrange(grid_size), random.randrange(grid_size)
        while is_terminal[i, j]:
            i, j = random.randrange(grid_size), random.randrange(grid_size)
        action = random.randrange(len(actions))
        episode_data = []

        while not is_terminal[i, j]:
            next_i, next_j = i + actions[action][0], j + actions[action][1]
            next_i, next_j, reward = get_reward(i, j, next_i, next_j)
            episode_data.append((i, j, action, reward))
            i, j = next_i, next_j
            if not is_terminal[i, j]:
                action = random.randrange(len(actions)) if random.random() < epsilon else np.argmax(Q[i, j])

        G = 0
        for i, j, action, reward in reversed(episode_data):
            G = gamma * G + reward
            returns_count[i, j, action] += 1
            Q[i, j, action] += (G - Q[i, j, action]) / returns_count[i, j, action]

    policy = np.argmax(Q, axis=2)
    return policy, Q

optimal_policy, Q_values = monte_carlo_epsilon_soft()

print("Optimal Policy (0=Up, 1=Down, 2=Left, 3=Right):")
print(optimal_policy)
print("\nAssociated Value Function:")
print(np.max(Q_values, axis=2))


KeyboardInterrupt: 

In [24]:
import numpy as np
import random

# Gridworld setup
grid_size = 5
gamma = 0.95  # Discount factor
actions = [(0, 1), (0, -1), (1, 0), (-1, 0)]  # Up, Down, Right, Left
epsilon = 0.1  # Exploration factor

# Initialize rewards for all states with -0.2 default (white-to-white transitions)
rewards = np.full((grid_size, grid_size), -0.2)
rewards[0, 1] = 5   # Blue square reward
rewards[0, 4] = 2.5 # Green square reward

# Terminal states setup
is_terminal = np.zeros((grid_size, grid_size), dtype=bool)
is_terminal[2, 4] = True  # Terminal state at location [2,4]
is_terminal[4, 0] = True  # Terminal state at location [4,0]

def get_reward(i, j, next_i, next_j):
    """Calculate and return the next state positions and reward."""
    if next_i < 0 or next_i >= grid_size or next_j < 0 or next_j >= grid_size:
        return i, j, -0.5  # Penalty for stepping off the grid
    if is_terminal[next_i, next_j]:
        return next_i, next_j, 0  # No further rewards when moving into a terminal state
    return next_i, next_j, rewards[next_i, next_j]  # Return the actual reward from the rewards matrix

def monte_carlo_epsilon_soft(num_episodes=500, epsilon=0.1, gamma=0.95):
    Q = np.zeros((grid_size, grid_size, len(actions)))
    returns_count = np.zeros((grid_size, grid_size, len(actions)))

    for episode in range(num_episodes):
        # Ensure starting state is not terminal
        while True:
            i, j = random.randrange(grid_size), random.randrange(grid_size)
            if not is_terminal[i, j]:
                break
        
        episode_data = []
        while not is_terminal[i, j]:
            action = random.randrange(len(actions)) if random.random() < epsilon else np.argmax(Q[i, j])
            next_i, next_j = i + actions[action][0], j + actions[action][1]
            next_i, next_j, reward = get_reward(i, j, next_i, next_j)
            episode_data.append((i, j, action, reward))
            i, j = next_i, next_j
        
        G = 0
        for i, j, action, reward in reversed(episode_data):
            G = gamma * G + reward
            returns_count[i, j, action] += 1
            Q[i, j, action] += (G - Q[i, j, action]) / returns_count[i, j, action]

    policy = np.argmax(Q, axis=2)
    return policy, Q

# Reduced number of episodes for faster runtime (for testing)
optimal_policy, Q_values = monte_carlo_epsilon_soft(num_episodes=100)

print("Optimal Policy (0=Up, 1=Down, 2=Left, 3=Right):")
print(optimal_policy)
print("\nAssociated Value Function:")
print(np.max(Q_values, axis=2))


Optimal Policy (0=Up, 1=Down, 2=Left, 3=Right):
[[0 1 1 1 1]
 [3 3 3 1 1]
 [3 3 3 3 0]
 [2 3 1 2 1]
 [0 1 3 1 3]]

Associated Value Function:
[[45.93805436 43.16767849 45.47753596 42.66545446 34.5278757 ]
 [43.18151144 45.6284717  42.67901964 40.05003104 36.25873461]
 [40.59214757 42.75957338 39.96061083 37.98457759  0.        ]
 [ 0.         39.60045647 34.47569908 24.06525325  4.15283234]
 [ 0.          0.         31.69450345 26.57017019  2.99042306]]


In [18]:
import numpy as np

# Initialize grid size and discount factor
size = 5
gamma = 0.95

# Initialize rewards and transitions with terminal states
def init_environment():
    rewards = np.full((size, size), -0.2)
    rewards[0, 1] = 5
    rewards[1, 2] = 2.5
    terminal_states = [(2, 2), (3, 3)]
    for (i, j) in terminal_states:
        rewards[i, j] = 0

    transitions = np.zeros((size, size, 4, 2), dtype=int)
    for i in range(size):
        for j in range(size):
            if (i, j) not in terminal_states:
                transitions[i, j, 0] = [max(i-1, 0), j]  # Up
                transitions[i, j, 1] = [min(i+1, size-1), j]  # Down
                transitions[i, j, 2] = [i, max(j-1, 0)]  # Left
                transitions[i, j, 3] = [i, min(j+1, size-1)]  # Right

    transitions[0, 1] = [[4, 1], [4, 1], [4, 1], [4, 1]]
    transitions[1, 2] = [[2, 3], [2, 3], [2, 3], [2, 3]]

    return rewards, transitions, terminal_states

rewards, transitions, terminal_states = init_environment()

# Policy iteration with importance sampling
def policy_iteration_importance_sampling(episodes=1000, gamma=0.95, epsilon=0.1):
    V = np.zeros((size, size))
    # Initialize policy with equal probability for all actions
    policy = np.full((size, size, 4), 0.25)
    C = np.zeros((size, size))

    def generate_episode(policy):
        state = (np.random.randint(0, size), np.random.randint(0, size))
        while state in terminal_states:
            state = (np.random.randint(0, size), np.random.randint(0, size))
        episode = []
        
        while state not in terminal_states:
            action = np.random.choice(4, p=policy[state[0], state[1]])
            next_state = transitions[state[0], state[1], action]
            reward = rewards[next_state[0], next_state[1]]
            episode.append((state, action, reward))
            state = tuple(next_state)
        
        return episode

    for _ in range(episodes):
        episode = generate_episode(policy)
        G = 0
        W = 1
        
        for t in range(len(episode)-1, -1, -1):
            state, action, reward = episode[t]
            G = gamma * G + reward
            C[state[0], state[1]] += W
            V[state[0], state[1]] += (W / C[state[0], state[1]]) * (G - V[state[0], state[1]])
            
            optimal_action = np.argmax([
                rewards[transitions[state[0], state[1], a][0], transitions[state[0], state[1], a][1]] + gamma * V[transitions[state[0], state[1], a][0], transitions[state[0], state[1], a][1]]
                for a in range(4)
            ])
            policy[state[0], state[1]] = np.full(4, epsilon / 4)
            policy[state[0], state[1]][optimal_action] += 1 - epsilon
            
            if action != optimal_action:
                break
            W *= 1.0 / policy[state[0], state[1]][action]
            
    return policy, V

policy_importance_sampling, V_importance_sampling = policy_iteration_importance_sampling()
print("Optimal Policy (Importance Sampling):")
print(policy_importance_sampling)
print("Value Function (Importance Sampling):")
print(V_importance_sampling)


Optimal Policy (Importance Sampling):
[[[0.25  0.25  0.25  0.25 ]
  [0.25  0.25  0.25  0.25 ]
  [0.25  0.25  0.25  0.25 ]
  [0.925 0.025 0.025 0.025]
  [0.25  0.25  0.25  0.25 ]]

 [[0.25  0.25  0.25  0.25 ]
  [0.925 0.025 0.025 0.025]
  [0.925 0.025 0.025 0.025]
  [0.025 0.025 0.925 0.025]
  [0.25  0.25  0.25  0.25 ]]

 [[0.925 0.025 0.025 0.025]
  [0.025 0.025 0.025 0.925]
  [0.25  0.25  0.25  0.25 ]
  [0.925 0.025 0.025 0.025]
  [0.925 0.025 0.025 0.025]]

 [[0.025 0.925 0.025 0.025]
  [0.025 0.025 0.025 0.925]
  [0.925 0.025 0.025 0.025]
  [0.25  0.25  0.25  0.25 ]
  [0.025 0.025 0.925 0.025]]

 [[0.25  0.25  0.25  0.25 ]
  [0.025 0.025 0.925 0.025]
  [0.925 0.025 0.025 0.025]
  [0.925 0.025 0.025 0.025]
  [0.925 0.025 0.025 0.025]]]
Value Function (Importance Sampling):
[[ 0.          0.          0.          1.9945      0.        ]
 [ 0.         -0.2        -0.2         1.1038961   0.        ]
 [-0.2        -0.02069416  0.          0.         -0.2       ]
 [-0.39       -0.2       

In [26]:
import numpy as np
import random

# Gridworld parameters
grid_size = 5
gamma = 0.95

# Reward structure
rewards = np.zeros((grid_size, grid_size))
rewards[0, 1] = 5  # Blue square
rewards[2, 3] = 2.5  # Green square
# More reward definitions as needed

# Transition probabilities
# We use a dictionary to hold transitions for special states
transitions = {
    (0, 1): (2, 3),  # Blue to Red
    # More transitions as needed
}

# Initialize value function and policy
Q = np.zeros((grid_size, grid_size, 4))
policy = np.ones((grid_size, grid_size, 4)) * 0.25  # Equiprobable random policy

# Actions
actions = [(-1, 0), (1, 0), (0, -1), (0, 1)]  # up, down, left, right

# Function to get next state
def get_next_state(state, action):
    next_state = (state[0] + action[0], state[1] + action[1])
    if next_state[0] < 0 or next_state[0] >= grid_size or next_state[1] < 0 or next_state[1] >= grid_size:
        return state
    return next_state

# Generate an episode using the behavior policy
def generate_episode(policy):
    episode = []
    state = (random.randint(0, grid_size - 1), random.randint(0, grid_size - 1))
    while len(episode) < 20:  # limit the length of the episode
        action_idx = np.random.choice(4, p=policy[state])
        action = actions[action_idx]
        next_state = get_next_state(state, action)
        reward = rewards[next_state]
        if state in transitions:
            next_state = transitions[state]
            reward = rewards[next_state]
        episode.append((state, action_idx, reward))
        state = next_state
    return episode

# Off-Policy Monte Carlo Control with Weighted Importance Sampling
def off_policy_mc_control(policy, Q, gamma, episodes=1000):
    C = np.zeros_like(Q)  # Cumulative sum of importance weights
    
    # Target policy initialization (deterministic)
    target_policy = np.zeros_like(policy)
    
    for i in range(grid_size):
        for j in range(grid_size):
            target_policy[i, j, np.argmax(Q[i, j])] = 1.0
    
    for _ in range(episodes):
        episode = generate_episode(policy)
        G = 0
        W = 1
        
        for (state, action_idx, reward) in reversed(episode):
            G = gamma * G + reward
            C[state][action_idx] += W
            Q[state][action_idx] += (W / C[state][action_idx]) * (G - Q[state][action_idx])
            target_policy[state] = np.eye(4)[np.argmax(Q[state])]
            if action_idx != np.argmax(target_policy[state]):
                break
            W /= policy[state][action_idx]
    
    # Update the policy to be greedy with respect to Q
    for i in range(grid_size):
        for j in range(grid_size):
            best_action = np.argmax(Q[i, j])
            policy[i, j] = np.eye(4)[best_action]
    
    return policy, Q

# Initialize policy and Q-values
policy = np.ones((grid_size, grid_size, 4)) * 0.25  # Equiprobable random policy
Q = np.zeros((grid_size, grid_size, 4))

# Learn the optimal policy
optimal_policy, Q_values = off_policy_mc_control(policy, Q, gamma)

# Print the learned optimal policy
print("Learned Optimal Policy:")
print(optimal_policy)

print("Q-values:")
print(Q_values)


Learned Optimal Policy:
[[[0. 0. 0. 1.]
  [0. 0. 0. 1.]
  [0. 0. 1. 0.]
  [0. 1. 0. 0.]
  [0. 1. 0. 0.]]

 [[0. 0. 0. 1.]
  [1. 0. 0. 0.]
  [1. 0. 0. 0.]
  [0. 1. 0. 0.]
  [0. 0. 1. 0.]]

 [[0. 1. 0. 0.]
  [1. 0. 0. 0.]
  [0. 0. 0. 1.]
  [0. 1. 0. 0.]
  [1. 0. 0. 0.]]

 [[1. 0. 0. 0.]
  [0. 0. 1. 0.]
  [0. 0. 0. 1.]
  [1. 0. 0. 0.]
  [0. 0. 1. 0.]]

 [[1. 0. 0. 0.]
  [1. 0. 0. 0.]
  [0. 0. 1. 0.]
  [0. 0. 1. 0.]
  [1. 0. 0. 0.]]]
Q-values:
[[[0.         0.         6.69253731 7.375     ]
  [2.5        2.5        3.45       4.50555556]
  [2.375      0.         8.4806383  0.        ]
  [5.25901235 8.19231265 5.23278689 4.03552472]
  [0.         4.17826616 0.68113208 0.        ]]

 [[0.         0.         0.         1.58333333]
  [8.8044186  8.5486484  0.         0.        ]
  [8.22179692 3.5815     5.76263736 1.35714286]
  [7.78689533 8.46351347 7.93913113 3.89675177]
  [0.         4.01638889 4.24945518 1.03142857]]

 [[2.40666667 3.83891318 3.4295     0.        ]
  [8.6761487  0.        

In [28]:
import numpy as np
import random

# Gridworld parameters
grid_size = 5
gamma = 0.95

# Reward structure
rewards = np.full((grid_size, grid_size), -0.2)  # All white squares
rewards[0, 1] = 5  # Blue square
rewards[2, 3] = 2.5  # Green square
# Adding terminal states
terminal_states = [(1, 1), (3, 3)]  # Example terminal states

# Transition probabilities
# We use a dictionary to hold transitions for special states
transitions = {
    (0, 1): (2, 3),  # Blue to Red
    # More transitions as needed
}

# Initialize value function and policy
Q = np.zeros((grid_size, grid_size, 4))
policy = np.ones((grid_size, grid_size, 4)) * 0.25  # Equiprobable random policy

# Actions
actions = [(-1, 0), (1, 0), (0, -1), (0, 1)]  # up, down, left, right

# Function to get next state
def get_next_state(state, action):
    next_state = (state[0] + action[0], state[1] + action[1])
    if next_state[0] < 0 or next_state[0] >= grid_size or next_state[1] < 0 or next_state[1] >= grid_size:
        return state
    return next_state

# Check if state is terminal
def is_terminal(state):
    return state in terminal_states

# Generate an episode using the behavior policy
def generate_episode(policy):
    episode = []
    state = (random.randint(0, grid_size - 1), random.randint(0, grid_size - 1))
    while not is_terminal(state):
        action_idx = np.random.choice(4, p=policy[state])
        action = actions[action_idx]
        next_state = get_next_state(state, action)
        reward = rewards[next_state]
        if state in transitions:
            next_state = transitions[state]
            reward = rewards[next_state]
        episode.append((state, action_idx, reward))
        state = next_state
    return episode

# Off-Policy Monte Carlo Control with Weighted Importance Sampling
def off_policy_mc_control(policy, Q, gamma, episodes=1000):
    C = np.zeros_like(Q)
    target_policy = np.zeros((grid_size, grid_size, 4))
    
    for i in range(grid_size):
        for j in range(grid_size):
            target_policy[i, j, np.argmax(Q[i, j])] = 1.0
    
    for _ in range(episodes):
        episode = generate_episode(policy)
        G = 0
        W = 1
        
        for (state, action_idx, reward) in reversed(episode):
            G = gamma * G + reward
            C[state][action_idx] += W
            Q[state][action_idx] += (W / C[state][action_idx]) * (G - Q[state][action_idx])
            target_policy[state] = np.eye(4)[np.argmax(Q[state])]
            if action_idx != np.argmax(target_policy[state]):
                break
            W /= policy[state][action_idx]
    
    # Update the policy to be greedy with respect to Q
    for i in range(grid_size):
        for j in range(grid_size):
            best_action = np.argmax(Q[i, j])
            policy[i, j] = np.eye(4)[best_action]
    
    return policy, Q

# Initialize policy and Q-values
policy = np.ones((grid_size, grid_size, 4)) * 0.25  # Equiprobable random policy
Q = np.zeros((grid_size, grid_size, 4))

# Learn the optimal policy using off-policy Monte Carlo control
optimal_policy, Q_values = off_policy_mc_control(policy, Q, gamma)

# Print the learned optimal policy
print("Learned Optimal Policy:")
print(optimal_policy)

print("Q-values:")
print(Q_values)


Learned Optimal Policy:
[[[1. 0. 0. 0.]
  [1. 0. 0. 0.]
  [1. 0. 0. 0.]
  [1. 0. 0. 0.]
  [1. 0. 0. 0.]]

 [[1. 0. 0. 0.]
  [1. 0. 0. 0.]
  [1. 0. 0. 0.]
  [1. 0. 0. 0.]
  [1. 0. 0. 0.]]

 [[1. 0. 0. 0.]
  [0. 1. 0. 0.]
  [1. 0. 0. 0.]
  [1. 0. 0. 0.]
  [1. 0. 0. 0.]]

 [[1. 0. 0. 0.]
  [1. 0. 0. 0.]
  [1. 0. 0. 0.]
  [1. 0. 0. 0.]
  [1. 0. 0. 0.]]

 [[1. 0. 0. 0.]
  [1. 0. 0. 0.]
  [1. 0. 0. 0.]
  [0. 1. 0. 0.]
  [1. 0. 0. 0.]]]
Q-values:
[[[ 0.   0.   0.   0. ]
  [ 0.   0.   0.   0. ]
  [ 0.   0.   0.   0. ]
  [ 0.   0.   0.   0. ]
  [ 0.   0.   0.   0. ]]

 [[ 0.   0.   0.  -0.2]
  [ 0.   0.   0.   0. ]
  [ 0.   0.  -0.2  0. ]
  [ 0.   0.   0.   0. ]
  [ 0.   0.   0.   0. ]]

 [[ 0.   0.   0.   0. ]
  [-0.2  0.   0.   0. ]
  [ 0.   0.   0.   0. ]
  [ 0.  -0.2  0.   0. ]
  [ 0.   0.   0.   0. ]]

 [[ 0.   0.   0.   0. ]
  [ 0.   0.   0.   0. ]
  [ 0.   0.   0.  -0.2]
  [ 0.   0.   0.   0. ]
  [ 0.   0.  -0.2  0. ]]

 [[ 0.   0.   0.   0. ]
  [ 0.   0.   0.   0. ]
  [ 0.   0.   0.   0

In [20]:
def policy_iteration_permuted_squares(episodes=500, gamma=0.95):
    V = np.zeros((size, size))
    policy = np.random.choice(4, size=(size, size))

    def permute_squares():
        if np.random.rand() < 0.5:
            return [(1, 2), (0, 1)], [(0, 1), (1, 2)]
        else:
            return [(0, 1), (1, 2)], [(1, 2), (0, 1)]
    
    for _ in range(episodes):
        blue_squares, green_squares = permute_squares()
        
        for _ in range(100):  # Adjust number of iterations
            policy, V = policy_iteration(V, policy)
            rewards[blue_squares[0][0], blue_squares[0][1]] = 5
            rewards[blue_squares[1][0], blue_squares[1][1]] = -0.2
            rewards[green_squares[0][0], green_squares[0][1]] = 2.5
            rewards[green_squares[1][0], green_squares[1][1]] = -0.2
        
    return policy, V

policy_permuted_squares, V_permuted_squares = policy_iteration_permuted_squares()
print("Optimal Policy (Permuted Squares):")
print(policy_permuted_squares)
print("Value Function (Permuted Squares):")
print(V_permuted_squares)


Optimal Policy (Permuted Squares):
[[0 0 1 2 2]
 [3 3 0 2 2]
 [0 0 0 0 0]
 [1 1 0 0 2]
 [1 2 3 0 2]]
Value Function (Permuted Squares):
[[-0.74345619 -1.13325016 -0.12854713 -0.42643028 -0.58721484]
 [-0.85251428 -0.46594164 -0.98157062 -0.23762868 -0.76983498]
 [-1.01936152 -0.99452445 -0.90628338 -0.82270592 -0.99277685]
 [-1.05085445 -1.16430589 -1.03248784 -0.90628338 -0.983627  ]
 [-0.85396306 -0.98236859 -0.94939194 -0.84526219 -0.85650213]]


In [21]:
# def policy_iteration_importance_sampling(V, policy, episodes=500, gamma=0.95):
#     def generate_episode(policy):
#         state = (np.random.randint(0, size), np.random.randint(0, size))
#         while state in terminal_states:
#             state = (np.random.randint(0, size), np.random.randint(0, size))
#         episode = []
        
#         while state not in terminal_states:
#             action = policy[state[0], state[1]]
#             next_state = transitions[state[0], state[1], action]
#             reward = rewards[next_state[0], next_state[1]]
#             episode.append((state, action, reward))
#             state = tuple(next_state)
        
#         return episode

#     def compute_importance_sampling_ratios(episode, policy, behavior_policy):
#         ratios = []
#         for state, action, reward in episode:
#             if behavior_policy[state[0], state[1]] == action:
#                 ratios.append(1.0)
#             else:
#                 ratios.append(0.0)
#         return np.prod(ratios)

#     for _ in range(episodes):
#         behavior_policy = np.random.choice(4, size=(size, size))
#         episode = generate_episode(behavior_policy)
#         G = 0
#         W = 1
        
#         for t in range(len(episode)-1, -1, -1):
#             state, action, reward = episode[t]
#             G = gamma * G + reward
#             V[state[0], state[1]] += (W / np.sum(W)) * (G - V[state[0], state[1]])
#             policy[state[0], state[1]] = np.argmax([
#                 rewards[transitions[state[0], state[1], a][0], transitions[state[0], state[1], a][1]] + gamma * V[transitions[state[0], state[1], a][0], transitions[state[0], state[1], a][1]]
#                 for a in range(4)
#             ])
#             W *= 1.0 / (0.25)  # Assume equiprobable behavior policy
            
#     return policy, V

# policy_importance_sampling = np.random.choice(4, size=(size, size))
# V_importance_sampling = np.zeros((size, size))
# policy_importance_sampling, V_importance_sampling = policy_iteration_importance_sampling(V_importance_sampling, policy_importance_sampling)
# print("Optimal Policy (Importance Sampling):")
# print(policy_importance_sampling)
# print("Value Function (Importance Sampling):")
# print(V_importance_sampling)
